In [1]:
from random import choices

import networkx as nx
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from models.observableSubgraph import create_subgraph, simulate_observable_nodes
from models.seeding import Seeding
from models.independentCascade import IndependentCascade

# Import NetHept Network Data

In [2]:
file = open('data/weic-graphdata/hep.txt', 'r')
first_line = file.readline()
meta = first_line.split()
num_nodes = int(meta[0])
num_edges = int(meta[1])
print(f"|V|: {num_nodes}, |E|: {num_edges}")

edges = [line.strip().split() for line in file.readlines()]
edges = [(int(e[0]), int(e[1])) for e in edges]

|V|: 15233, |E|: 58891


In [3]:
# create a directed graph object
# note: there are duplicated edges. If you want to keep the duplicates, use MultiDiGraph instead
graph = nx.DiGraph()
graph.add_edges_from(edges)
print(f"Number of nodes: {graph.number_of_nodes()}\nNumber of edges: {graph.number_of_edges()}")

Number of nodes: 15233
Number of edges: 32235


# Simulate Information Spreading in NetHept

## Create an observable subgraph with visibility of 10%

In [4]:
target_visibility = 0.1
fully_observ_O, boundary_nodes = simulate_observable_nodes(graph, target_visibility)
observable_nodes = fully_observ_O + boundary_nodes
observable_graph = create_subgraph(graph, observable_nodes)

## Select 5 initial seeds using weigted degree-based heuristic

In [5]:
seeding = Seeding(observable_graph, method = 'weighted-degree-based')
seeds = seeding(boundary_nodes, weight = 3, num_seed = 5)
seeds

[196, 326, 474, 525, 80]

# Simulate Weighted Independent Casecade Model

In [6]:
ic_model = IndependentCascade(graph, seeds)
activated = ic_model.simulate()
print(ic_model.get_spread())

118


## Run multiple simulations of the IC model

In [7]:
avg_spread = ic_model.run_simulations(iters = 10)
print(f"Average spread across 10 runs: {avg_spread}")

Average spread across 10 runs: 97.6
